In [ ]:
import os
import sys
sys.path.insert(0,"..")
import numpy as np
import matplotlib.pyplot as plt
import pyemu
print(pyemu.__file__)
import flopy
import platform
from pathlib import Path
import shutil
import pandas as pd

In [ ]:
t_d = "template"
if not os.path.exists(t_d):
    raise Exception("need to run simple_pstfrom notebook")

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"at.pst"))
pst.npar,pst.nobs

In [ ]:
pst.pestpp_options

In [ ]:
if "ies_par_en" not in pst.pestpp_options:
    raise Exception("prior par ensemble not found")

In [ ]:
pst.pestpp_options = {"ies_par_en":pst.pestpp_options["ies_par_en"]}

In [ ]:
pst.control_data.noptmax = 3

In [ ]:
pst.obs_groups

In [ ]:
weighted_groups = [i for i in pst.obs_groups if i.endswith('group')]
weighted_groups

In [ ]:
wts = [.4,.3,.2,.2]

In [ ]:
with open(f'{t_d}/phi_factors.csv', 'w') as ofp:
    [ofp.write(f'{g},{w}\n') for g,w in zip(weighted_groups,wts)]

In [ ]:
pst.pestpp_options['ies_phi_factor_file']='phi_factors.csv'

In [ ]:
pst.write(os.path.join(t_d,"at.pst"),version=2)

In [ ]:
m_d = "master_ies_simple"

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-ies","at.pst",num_workers=8,master_dir=m_d,worker_root='.')

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"at.pst"))

In [ ]:
oe = pst.ies.obsen
oe

In [ ]:
nzoe = oe.loc[:,pst.nnz_obs_names]
noise = pst.ies.noise
itrs = pst.ies.phiactual.iteration.values
itrs

In [ ]:
obs = pst.observation_data
nzobs = obs.loc[obs.weight > 0,:]

In [ ]:
odict = {n:v for n,v in zip(nzobs.obsnme,nzobs.obsval)}

In [ ]:
ptoe = pst.ies.get("obsen{0}".format(itrs.max()))
proe = pst.ies.obsen0

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))
names = obs.loc[obs.obgnme=="headgroup","obsnme"].values
for oname in names:
    if "at.csv" not in oname:
        continue
    ovals = [odict[oname] for _ in range(noise.shape[0])]
    ax.scatter(ovals,noise[oname],marker='.',c='r',s=100,alpha=0.25)
    
    ovals = [odict[oname] for _ in range(ptoe.shape[0])]
    ax.scatter(ovals,ptoe[oname],marker='.',c='b',alpha=0.5)
    #ylim = ax.get_ylim()
    ovals = [odict[oname] for _ in range(proe.shape[0])]
    ax.scatter(ovals,proe[oname],marker='.',c='0.5',alpha=0.5,zorder=0)
    #ax.set_ylim(ylim)
    #ax.set_xlim(ylim)
mn,mx = noise.loc[:,names].values.min(),noise.loc[:,names].values.max()
ax.plot([mn,mx],[mn,mx],"k--",lw=3)
ax.set_xlabel("obs")
ax.set_ylabel("sim")
ax.set_xlim(mn,mx)

_ = ax.set_ylim(mn,mx)

    

In [ ]:
obs.loc[obs.obgnme=="rivgroup","obsnme"].values

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))
names = obs.loc[obs.obgnme=="rivgroup","obsnme"].values
for oname in names:

    ovals = [odict[oname] for _ in range(noise.shape[0])]
    ax.scatter(ovals,noise[oname],marker='.',c='r',s=100,alpha=0.25)
    
    ovals = [odict[oname] for _ in range(ptoe.shape[0])]
    ax.scatter(ovals,ptoe[oname],marker='.',c='b',alpha=0.5)
    #ylim = ax.get_ylim()
    ovals = [odict[oname] for _ in range(proe.shape[0])]
    ax.scatter(ovals,proe[oname],marker='.',c='0.5',alpha=0.5,zorder=0)
    #ax.set_ylim(ylim)
    #ax.set_xlim(ylim)
mn,mx = noise.loc[:,names].values.min(),noise.loc[:,names].values.max()
ax.plot([mn,mx],[mn,mx],"k--",lw=3)
ax.set_xlabel("obs")
ax.set_ylabel("sim")
ax.set_xlim(mn,mx)

_ = ax.set_ylim(mn,mx)

In [ ]:
kobs = obs.loc[obs.obsnme.str.contains("k_aq"),:].copy()
kobs['i'] = kobs.i.astype(int)
kobs['j'] = kobs.j.astype(int)

In [ ]:
prarr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
prarr[kobs.i,kobs.j] = np.log10(ptoe.loc[:,kobs.obsnme].mean())
ptarr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
ptarr[kobs.i,kobs.j] = np.log10(ptoe.loc["base",kobs.obsnme])
fig,axes = plt.subplots(1,2,figsize=(8,4))
cb = axes[0].imshow(prarr,vmin=ptarr.min())
plt.colorbar(cb,ax=axes[0])
axes[0].set_title("mean")

cb = axes[1].imshow(ptarr,vmax=prarr.max())
plt.colorbar(cb,ax=axes[1])
axes[1].set_title("min err var")


In [ ]:
prarr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
prarr[kobs.i,kobs.j] = np.log10(proe.loc[:,kobs.obsnme].std())
ptarr = np.zeros((kobs.i.max()+1,kobs.j.max()+1))
ptarr[kobs.i,kobs.j] = np.log10(ptoe.loc[:,kobs.obsnme].std())
fig,axes = plt.subplots(1,2,figsize=(8,4))
cb = axes[0].imshow(prarr,vmin=ptarr.min())
plt.colorbar(cb,ax=axes[0])
axes[0].set_title("prior stdev")
cb = axes[1].imshow(ptarr,vmax=prarr.max())
plt.colorbar(cb,ax=axes[1])
axes[1].set_title("posterior stdev")

In [ ]:
bobs = obs.loc[obs.oname=="at.budget.csv",:]
for oname in bobs.obsnme:
    fig,ax = plt.subplots(1,1,figsize=(6,6))
    ax.hist(proe.loc[:,oname],bins=10,fc="0.5",alpha=0.5,density=True)
    ax.hist(ptoe.loc[:,oname],bins=10,fc="b",alpha=0.5,density=True)
    ax.set_title(oname)
    ax.set_yticks([])
    